# ThoraricSurgery.csv 분석하기

- 1.데이터 전처리
- 2.학습용 데이터와 테스트용 데이터 분리
- 3.학습 시키기
- 4.예측 하기
- 5.평가 - 예측값과 실제값을 비교해서 모델 성능 평가
- 6.grid Cv


- 1. 데이터 전처리

In [1]:
import pandas as pd
import numpy as np

In [2]:
ThoraricsSurgery_df = pd.read_csv('ThoraricSurgery.csv',header=None)

In [3]:
ThoraricsSurgery_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,98,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76,0
466,369,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77,0
467,406,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62,0
468,25,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58,1


In [4]:
X = ThoraricsSurgery_df.iloc[:,:-1].values
y = ThoraricsSurgery_df.iloc[:,[-1]].values

In [5]:
type(X),type(y),len(X),len(y)

(numpy.ndarray, numpy.ndarray, 470, 470)

In [6]:
X.shape,y.shape

((470, 17), (470, 1))

- 2.학습용 데이터와 테스트용 데이터 분리

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(
    X,y,random_state=2021,stratify=y,test_size=0.2
)

In [9]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((376, 17), (94, 17), (376, 1), (94, 1))

- 3.학습 시키기

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=2021)
dtc.get_params()

In [11]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier(random_state=2021)

- 4. 학습된 모델로 테스트 데이터 예측

In [12]:
pred = dtc.predict(X_test)
pred

array([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0], dtype=int64)

# 5.모델 평가

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.6808510638297872

In [ ]:
from sklearn.svm import SVC
svc = SVC(random_state=2021)
svc.fit(X_train,y_train)

In [15]:
pred_sv = svc.predict(X_test)
accuracy_score(y_test, pred_sv)

0.851063829787234

In [ ]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2021)
lrc.fit(X_train, y_train)

In [ ]:
# 해결책1 - max_iter 하이퍼 파라메터 값 조정
lrc2 = LogisticRegression(random_state=2021, max_iter=1000)
lrc2.fit(X_train, y_train)

In [37]:
pred_lr2 = lrc2.predict(X_test)
accuracy_score(y_test, pred_lr2)

0.8404255319148937

최적의 값 구하기

In [16]:
from sklearn.model_selection import GridSearchCV

In [17]:
params = {
    'max_depth' : [2,3,4,5,6],
    'min_samples_split': [2,3,4]
}

In [25]:
grid_dt = GridSearchCV(dtc, param_grid=params, scoring='accuracy', cv=3)
grid_dt.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [41]:
grid_dt.best_score_,grid_dt.best_params_

(0.8324867724867725, {'max_depth': 2, 'min_samples_split': 2})

In [28]:
best_dt =grid_dt.best_estimator_

In [29]:
best_dt.score(X_test,y_test)

0.8297872340425532

오차행렬

In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[61, 19],
       [11,  3]], dtype=int64)

precision_score,recall_score

In [40]:
from sklearn.metrics import precision_score , recall_score
precision_score(y_test, pred), recall_score(y_test, pred)

(0.13636363636363635, 0.21428571428571427)

In [42]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, pred)
print(f'F1 스토어: {f1:.4f}')